# Processamento digital de imagens

## Atividade 3

 Considere a imagem dispon´ıvel neste link: https://www.dropbox.com/s/smg590xgajl13z9/42.png?dl=1
Leia atentamente as instru¸c˜oes no in´ıcio desta atividade. Ser˜ao avaliadas as suas competˆencias e habilidades em Processamento Digital de Imagens em tentar segmentar a imagem do link anterior. Deve-se evitar recortar manualmente a imagem
para tentar facilitar a segmenta¸c˜ao. Quando a imagem tiver objetos geom´etricos, incluir em seu interior a ´area (n´umero
de pixels) e escrever o tipo do objeto. Por exemplo, c´ırculo, retˆangulo, etc. N˜ao precisa pintar os objetos. Sugest˜ao:
utilizar a fun¸c˜ao cv2.findContours ap´os alguns pr´e-processamentos. Caso tenha alguma d´uvida, envie um email para
fzampirolli@ufabc.edu.br.

Veja como ler e mostrar a imagem da URL:


In [1]:
#Video: https://youtu.be/pX3tilBfqmM

#Bibliotecas
import matplotlib.pyplot as plt
from skimage import io
import numpy as np
import cv2 as cv2

#Ler imagem
img = io.imread("https://www.dropbox.com/s/smg590xgajl13z9/42.png?dl=1")
  
#Convertendo a imagem numa escala de cinza
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  
#Extraindo o limiar da escala de cinza
_, threshold = cv2.threshold(gray, 160 , 255, cv2.CHAIN_APPROX_NONE)
  
#Extraindo os contornos apartir do limiar
contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
  
i = 0
  
#Para cada contorno
for contour in contours:
    #Ignora o primeiro contorno, porque ele considera a borda da imagem como um
    if i == 0:
        i = 1
        continue

    #Seta o meio da forma/figura para escrever dentro
    M = cv2.moments(contour)
    if M['m00'] != 0.0:
        x = int(M['m10']/M['m00'])
        y = int(M['m01']/M['m00'])
    
    #Calcula a area da forma e escreve no meio dela
    area = cv2.contourArea(contour)
    cv2.putText(img, "{:5.0f}".format(area), (x-20, y+9), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1)
  
    #Função especifica do OpenCV para aproximar a alguma forma
    approx = cv2.approxPolyDP(contour, 0.1 * cv2.arcLength(contour, True), True)
    
    #Se a aproximação for 3: escreve triangulo
    if len(approx) == 3:
        cv2.putText(img, 'triangulo', (x-20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1)
    elif len(approx) == 4: #Se a aproximação for 4 ele considera a proporção da forma/figura
        _, _, w, h = cv2.boundingRect(approx)
        aspectRatio = float(w)/h
        if aspectRatio >= 0.96 and aspectRatio < 1.04: #Se for bem proximo de simetrico é um circulo
            cv2.putText(img, 'circulo', (x-20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1)
        elif aspectRatio >= 0.80 and aspectRatio < 1.20: #Se for um pouco menos proximo de simetrico é um quadrado
            cv2.putText(img, 'quadrado', (x-20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1)
        else: #De resto um retangulo
            cv2.putText(img, 'retangulo', (x-20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1)
        #Esses elif não faz sentido teoricamente, mas foi o que funcionou na prática,
        # provavelmente por causa da baixa resulução da imagem
  
#Plota a imagem usando o Mat Plot
plt.figure(figsize = (200,200))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()